In [1]:
pip install tensorflow-datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

raw_train, raw_test = tfds.load(
    'cats_vs_dogs',
    split=["train[:80%]", "train[20%:]"],
    as_supervised =True
)

In [4]:
# FeaturesDict({
#     'image': Image(shape=(None, None, 3), dtype=tf.unit8),
#     'image/filename': Text(shape=(), dtype=tf.string),
#     'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
# })

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.image import ResizeMethod

In [8]:
def preprocess(image, label):
    out_image = tf.image.resize(image, [224,224], method = ResizeMethod.BICUBIC)
    out_image = tf.keras.applications.mobilenet_v2.preprocess_input(out_image)
    return out_image, label
batch_size = 32
train_batch = raw_train.map(preprocess).batch(batch_size)
test_batch = raw_test.map(preprocess).batch(batch_size)

In [9]:
mobilenet_base = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    weights="imagenet",
    include_top = False)

9406464/9406464 [==============================] - 2s 0us/step


In [10]:
mobilenet_base.trainable=False

mobilenet_model = tf.keras.Sequential([
    mobilenet_base,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1)
])

mobilenet_model.compile(optimizer="adam",
                        loss = "binary_crossentropy",
                        metrics=["accuracy"])

mobilenet_model.fit(train_batch, epochs=5)
mobilenet_model.evaluate(test_batch, verbose=2)

Epoch 1/5
582/582 [==============================] - 308s 525ms/step - loss: 0.4223 - accuracy: 0.9530
Epoch 2/5
582/582 [==============================] - 308s 530ms/step - loss: 0.2569 - accuracy: 0.9746
Epoch 3/5
582/582 [==============================] - 307s 528ms/step - loss: 0.2181 - accuracy: 0.9804
Epoch 4/5
582/582 [==============================] - 306s 526ms/step - loss: 0.2556 - accuracy: 0.9780
Epoch 5/5
582/582 [==============================] - 306s 525ms/step - loss: 0.1481 - accuracy: 0.9865
582/582 - 307s - loss: 0.1399 - accuracy: 0.9873 - 307s/epoch - 528ms/step


[0.13993947207927704, 0.9873186349868774]